In [1]:
!pip install datasets
!pip install transformers
!pip install wandb
!pip install llama-recipes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 76.8 MB/s 

In [2]:
import datasets
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
import torch
print(torch.cuda.is_available())

True


In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "openai-community/gpt2",
    device_map="cuda",
    use_cache=None,
    attn_implementation=None,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
model.device

device(type='cuda', index=0)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
user_prompt = '''A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?'''
batch = tokenizer(user_prompt, return_tensors="pt")

In [8]:
print("The batch keys are:",batch.keys())
print("The input_ids are:",batch["input_ids"])
print("Then length of the input_ids are:",len(batch["input_ids"][0]))

The batch keys are: dict_keys(['input_ids', 'attention_mask'])
The input_ids are: tensor([[   32, 33192,  2753,   362, 24667,   286,  4171, 13608,   290,  2063,
           326,   881,  2330, 13608,    13,  1374,   867, 24667,   287,  2472,
           857,   340,  1011,    30]])
Then length of the input_ids are: 24


In [9]:
batch = {k: v.to("cuda") for k, v in batch.items()}
with torch.no_grad():
    outputs = model.generate(
        **batch,
        max_new_tokens=50, #The maximum numbers of tokens to generate
        do_sample=False, #Whether or not to use sampling ; use greedy decoding otherwise.
        top_p=1.0, # [optional] If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
        temperature=0, # [optional] The value used to modulate the next token probabilities.
        min_length=None, #The minimum length of the sequence to be generated, input prompt + min_new_tokens
        use_cache=True, #[optional] Whether or not the model should use the past last key/values attentions Whether or not the model should use the past last key/values attentions (if applicable to the model) to speed up decoding.
        top_k=50, # [optional] The number of highest probability vocabulary tokens to keep for top-k-filtering.
        repetition_penalty=1.0, #The parameter for repetition penalty. 1.0 means no penalty.
        length_penalty=1, #[optional] Exponential penalty to the length that is used with beam-based generation.
        output_hidden_states= True, return_dict_in_generate=True,
    )
batch = {k: v.to("cpu") for k, v in batch.items()}

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [10]:
output_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

In [11]:
print("The complete decode:",output_text)
print("--------------------------------")
print("Just the model's response",output_text[len(user_prompt):])

The complete decode: A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?

The answer is that it takes about 1.5 bolts of blue fiber.

The reason for this is that the blue fiber is not as dense as the white fiber.

The reason for this is that the blue fiber is not
--------------------------------
Just the model's response 

The answer is that it takes about 1.5 bolts of blue fiber.

The reason for this is that the blue fiber is not as dense as the white fiber.

The reason for this is that the blue fiber is not


##Training

In [12]:
from dataclasses import dataclass
import wandb
from typing import List, Optional, Tuple, Union
from torch.utils.data import Dataset
from tqdm import tqdm
from transformers import default_data_collator
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from contextlib import nullcontext
from datetime import datetime
from llama_recipes.utils.memory_utils import MemoryTrace
import time
import os
import contextlib
import json
from transformers.data import DataCollatorForSeq2Seq

/usr/local/lib/python3.10/dist-packages/llama_recipes/model_checkpointing/checkpoint_handler.py:17: DeprecationWarning: `torch.distributed._shard.checkpoint` will be deprecated, use `torch.distributed.checkpoint` instead
  from torch.distributed._shard.checkpoint import (


In [13]:
def save_to_json(output_filename, train_step_loss, train_epoch_loss, train_step_ppl, train_epoch_ppl, val_step_loss, val_epoch_loss, val_step_ppl, val_epoch_ppl):
    metrics_data = {
        "train_step_loss": train_step_loss,
        "train_epoch_loss": train_epoch_loss,
        "train_step_perplexity": train_step_ppl,
        "train_epoch_perplexity": train_epoch_ppl,
        "val_step_loss": val_step_loss,
        "val_epoch_loss": val_epoch_loss,
        "val_step_perplexity": val_step_ppl,
        "val_epoch_perplexity": val_epoch_ppl
    }
    with open(output_filename, "w") as f:
        json.dump(metrics_data, f)

In [14]:
class LengthBasedBatchSampler(torch.utils.data.BatchSampler):
    def __init__(self, data_source, batch_size: int, drop_last: bool, shuffle: bool=True) -> None:
        if isinstance(next(iter(data_source)), dict):
            first_key = next(iter(next(iter(data_source)).keys()))
            self.lengths = [len(d[first_key]) for d in data_source]
        else:
            self.lengths = [len(d) for d in data_source]
        self.batch_size = batch_size
        self.drop_last = drop_last
        self.shuffle = shuffle

    def __iter__(self):
        ids = np.argsort(self.lengths, kind='mergesort')
        if self.drop_last:
            ids = ids[:len(ids) // self.batch_size * self.batch_size]

        batches = [ids[i:i+self.batch_size] for i in range(0, len(ids), self.batch_size)]

        if self.shuffle:
            random.shuffle(batches)

        for b in batches:
            yield b

    def __len__(self):
        if self.drop_last:
            return len(self.lengths) // self.batch_size
        else:
            return len(self.lengths) // self.batch_size + (len(self.lengths) % self.batch_size > 0)

In [15]:
def get_dataloader_kwargs(train_config, dataset, tokenizer, mode):
        kwargs = {}
        batch_size = train_config.batch_size_training if mode=="train" else train_config.val_batch_size
        if train_config.batching_strategy == "padding":
            kwargs["batch_sampler"] = LengthBasedBatchSampler(dataset, batch_size, drop_last=True, shuffle=mode=="train")
            kwargs["collate_fn"] = DataCollatorForSeq2Seq(tokenizer)
        elif train_config.batching_strategy == "packing":
            kwargs["batch_size"] = batch_size
            kwargs["drop_last"] = True
            kwargs["collate_fn"] = default_data_collator
        else:
            raise ValueError(f"Unknown batching strategy: {train_config.batching_strategy}")

        return kwargs

In [16]:
@contextlib.contextmanager
def profile(cfg, local_rank=None):
    use_profiler: bool = cfg.use_profiler
    use_flop_counter: bool = cfg.flop_counter
    if use_flop_counter and use_profiler:
        raise ValueError("Cannot use both profiler and flop counter")
    if use_profiler:
        # profiler needs a warmup stage to get the accurate profiling results
        wait_step, warmup_step, active_step = 1, 2, 3
        min_step = wait_step + warmup_step + active_step + 1
        if cfg.max_train_step > 0 and cfg.max_train_step < min_step:
            raise ValueError(f"pytorch profiler requires at least {min_step} train steps to finish the warm-up and recording stage, {wait_step} for wait_step, {warmup_step} for warmup_step, {active_step} for profiling step, please increase the max_train_step, current max_train_step {cfg.max_train_step}")
        print(f"pytorch profiling is activated and results will be saved in {cfg.profiler_dir}")
        with torch.profiler.profile(
            activities=[
                torch.profiler.ProfilerActivity.CPU,
                torch.profiler.ProfilerActivity.CUDA,
            ],
            schedule=torch.profiler.schedule(wait=wait_step, warmup=warmup_step, active=active_step, repeat=1),
            on_trace_ready=torch.profiler.tensorboard_trace_handler(
                cfg.profiler_dir
            ),
            profile_memory=True,
            with_stack=False,
            with_flops=True,
            record_shapes=True,
        ) as torch_profiler:
            yield torch_profiler
    elif use_flop_counter:
        if cfg.max_train_step > 0 and cfg.max_train_step <= cfg.flop_counter_start:
            raise ValueError(f"flop counter requires at least {cfg.flop_counter_start + 1} train steps, please increase the max_train_step, current max_train_step {cfg.max_train_step}")
        with FlopMeasure(rank=local_rank,warmup_step=cfg.flop_counter_start) as flop_counter:
            yield flop_counter
    else:
        torch_profiler = contextlib.nullcontext()
        yield None

In [17]:
class ConcatDataset(Dataset):
    def __init__(self, dataset, chunk_size=4096):
        self.dataset = dataset
        self.chunk_size = chunk_size

        self.samples = []
        buffer = {
            "input_ids": [],
            "attention_mask": [],
            "labels": []
            }

        for sample in tqdm(self.dataset, desc="Preprocessing dataset", dynamic_ncols=True):
            buffer = {k: v + sample[k] for k,v in buffer.items()}

            while len(next(iter(buffer.values()))) > self.chunk_size:
                self.samples.append({k: v[:self.chunk_size] for k,v in buffer.items()})
                buffer = {k: v[self.chunk_size:] for k,v in buffer.items()}

    def __getitem__(self, idx):
        return self.samples[idx]

    def __len__(self):
        return len(self.samples)

In [18]:
def train(model, train_dataloader, eval_dataloader, tokenizer, optimizer, lr_scheduler, gradient_accumulation_steps, train_config, wandb_run=None):
    """
    Trains the model on the given dataloader

    Args:
        model: The model to be trained
        train_dataloader: The dataloader containing the training data
        optimizer: The optimizer used for training
        lr_scheduler: The learning rate scheduler
        gradient_accumulation_steps: The number of steps to accumulate gradients before performing a backward/update operation
        num_epochs: The number of epochs to train for
        local_rank: The rank of the current node in a distributed setting
        train_config: The training configuration
        eval_dataloader: The dataloader containing the eval data
        tokenizer: tokenizer used in the eval for decoding the predicitons

    Returns: results dictionary containing average training and validation perplexity and loss
    """

    local_rank=None
    rank=None

    autocast = nullcontext
    train_prep = []
    train_loss = []
    val_prep = []
    val_loss =[]

    if train_config.save_metrics:
        metrics_filename = f"{train_config.output_dir}/metrics_data_{local_rank}-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
        train_step_perplexity = []
        train_step_loss = []
        val_step_loss = []
        val_step_perplexity = []

    epoch_times = []
    checkpoint_times = []
    results = {}
    best_val_loss = float("inf")
    total_train_steps = 0
    max_steps_reached = False  # Flag to indicate max training steps reached
    # Start the training loop
    for epoch in range(train_config.num_epochs):
        # stop when the maximum number of training steps is reached

        if max_steps_reached:
            break
        epoch_start_time = time.perf_counter()
        with MemoryTrace() as memtrace:  # track the memory usage
            model.train()
            total_loss = 0.0
            total_loss_f = 0.0
            total_loss_m = 0.0
            total_length = len(train_dataloader)//gradient_accumulation_steps
            pbar = tqdm(colour="blue", desc=f"Training Epoch: {epoch+1}", total=total_length, dynamic_ncols=True)
            with profile(train_config,local_rank) as profile_context:
                for step, batch in enumerate(train_dataloader):
                    total_train_steps += 1
                    # stop when the maximum number of training steps is reached
                    if train_config.max_train_step > 0 and total_train_steps > train_config.max_train_step:
                        max_steps_reached = True
                        print("max training steps reached, stopping training, total train steps finished: ", total_train_steps-1)
                        break
                    # print(f"The keys in the batch are:{batch.keys()}")
                    for key in batch.keys():
                        # if is_xpu_available():
                        #     batch[key] = batch[key].to('xpu:0')
                        # else:
                          batch[key] = batch[key].to('cuda:0')
                    with autocast():
                        outputs = model(**batch)
                        loss = outputs.loss

                    loss = loss / gradient_accumulation_steps

                    if train_config.save_metrics:
                        train_step_loss.append(loss.detach().float().item())
                        train_step_perplexity.append(float(torch.exp(loss.detach().float())))

                    total_loss += loss.detach().float()
                    loss.backward()

                    if (step + 1) % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
                        if train_config.gradient_clipping and train_config.gradient_clipping_threshold > 0.0:
                            torch.nn.utils.clip_grad_norm_(model.parameters(), train_config.gradient_clipping_threshold)
                        optimizer.step()
                        optimizer.zero_grad()
                        pbar.update(1)
                    if train_config.use_profiler or train_config.flop_counter:
                        profile_context.step()
                    if train_config.flop_counter and profile_context.is_done():
                        TFlops = profile_context.get_flops_per_sec() / 1e12
                    if wandb_run:
                        wandb_run.log({
                            'train/epoch': epoch + 1,
                            'train/step': epoch * len(train_dataloader) + step,
                            'train/loss': loss.detach().float(),
                        })

                    pbar.set_description(f"Training Epoch: {epoch+1}/{train_config.num_epochs}, step {step}/{len(train_dataloader)} completed (loss: {loss.detach().float()})")

                    if step % 50 == 0:
                        if train_config.run_validation:
                            eval_ppl, eval_epoch_loss, temp_val_loss, temp_step_perplexity = evaluation(model, train_config, eval_dataloader, local_rank, tokenizer, wandb_run)
                            if train_config.save_metrics:
                                val_step_loss.extend(temp_val_loss)
                                val_step_perplexity.extend(temp_step_perplexity)
                        model.train()

                    if train_config.save_metrics:
                        save_to_json(metrics_filename, train_step_loss, train_loss, train_step_perplexity, train_prep, val_step_loss, val_loss, val_step_perplexity, val_prep)
                pbar.close()

        epoch_end_time = time.perf_counter()-epoch_start_time
        epoch_times.append(epoch_end_time)
        train_epoch_loss = total_loss / len(train_dataloader)
        train_perplexity = torch.exp(train_epoch_loss)
        train_prep.append(float(train_perplexity))
        train_loss.append(float(train_epoch_loss))
        memtrace.print_stats()
        lr_scheduler.step()

        if train_config.run_validation:
            eval_ppl, eval_epoch_loss, temp_val_loss, temp_step_perplexity = evaluation(model, train_config, eval_dataloader, local_rank, tokenizer, wandb_run)
            if train_config.save_metrics:
                val_step_loss.extend(temp_val_loss)
                val_step_perplexity.extend(temp_step_perplexity)

            checkpoint_start_time = time.perf_counter()

            if train_config.save_model :
                epoch_dir = os.path.join(train_config.output_dir, f"epoch{epoch}")
                os.makedirs(epoch_dir, exist_ok=True)
                # Save the model in the new directory
                model.save_pretrained(epoch_dir)
                print(f"Model is saved in {epoch_dir} directory")

            checkpoint_end_time = time.perf_counter() - checkpoint_start_time
            checkpoint_times.append(checkpoint_end_time)
            if eval_epoch_loss < best_val_loss:
                best_val_loss = eval_epoch_loss
                print(f"best eval loss on epoch {epoch+1} is {best_val_loss}")
            val_loss.append(float(best_val_loss))
            val_prep.append(float(eval_ppl))
        print(f"Epoch {epoch+1}: train_perplexity={train_perplexity:.4f}, train_epoch_loss={train_epoch_loss:.4f}, epoch time {epoch_end_time}s")

        # Saving the results every epoch to plot later
        if train_config.save_metrics:
            save_to_json(metrics_filename, train_step_loss, train_loss, train_step_perplexity, train_prep, val_step_loss, val_loss, val_step_perplexity, val_prep)

    avg_epoch_time = sum(epoch_times)/ len(epoch_times)
    avg_checkpoint_time = sum(checkpoint_times)/ len(checkpoint_times) if len(checkpoint_times) > 0 else 0
    avg_train_prep = sum(train_prep)/len(train_prep)
    avg_train_loss = sum(train_loss)/len(train_loss)
    if train_config.run_validation:
        avg_eval_prep = sum(val_prep)/len(val_prep)
        avg_eval_loss = sum(val_loss)/len(val_loss)

    results['avg_train_prep'] = avg_train_prep
    results['avg_train_loss'] = avg_train_loss
    if train_config.run_validation:
        results['avg_eval_prep'] = avg_eval_prep
        results['avg_eval_loss'] = avg_eval_loss
    results["avg_epoch_time"] = avg_epoch_time
    results["avg_checkpoint_time"] = avg_checkpoint_time
    if train_config.save_metrics:
        results["metrics_filename"] = metrics_filename
    return results

In [19]:
def evaluation(model, train_config, eval_dataloader, local_rank, tokenizer, wandb_run):
    """
    Evaluates the model on the given dataloader

    Args:
        model: The model to evaluate
        eval_dataloader: The dataloader containing the evaluation data
        local_rank: The rank of the current node in a distributed setting
        tokenizer: The tokenizer used to decode predictions

    Returns: eval_ppl, eval_epoch_loss
    """
    model.eval()
    eval_preds = []
    val_step_loss = []
    val_step_perplexity = []
    eval_loss = 0.0  # Initialize evaluation loss
    eval_loss_f = 0.0
    eval_loss_m = 0.0
    total_eval_steps = 0
    with MemoryTrace() as memtrace:
        for step, batch in enumerate(tqdm(eval_dataloader,colour="green", desc="evaluating Epoch", dynamic_ncols=True)):
            total_eval_steps += 1
            # stop when the maximum number of eval steps is reached
            if train_config.max_eval_step > 0 and total_eval_steps > train_config.max_eval_step:
                break
            for key in batch.keys():
                # if is_xpu_available():
                #     batch[key] = batch[key].to('xpu:0')
                # else:
                batch[key] = batch[key].to('cuda:0')
            # Ensure no gradients are computed for this scope to save memory
            with torch.no_grad():
                # Forward pass and compute loss
                outputs = model(**batch)
                loss = outputs.loss
                if train_config.save_metrics:
                    val_step_loss.append(loss.detach().float().item())
                    val_step_perplexity.append(float(torch.exp(loss.detach().float())))

                eval_loss += loss.detach().float()
            # Decode predictions and add to evaluation predictions list
            preds = torch.argmax(outputs.logits, -1)
            eval_preds.extend(
                tokenizer.batch_decode(preds.detach().cpu().numpy(), skip_special_tokens=True)
            )
# Compute average loss and perplexity
    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_epoch_loss_f = eval_loss_f / len(eval_dataloader)
    eval_epoch_loss_m = eval_loss_m / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
# Print evaluation metrics
    if wandb_run:
        wandb_run.log({
                        'eval/perplexity': eval_ppl,
                        'eval/loss': eval_epoch_loss,
                    }, commit=False)
    return eval_ppl, eval_epoch_loss, val_step_loss, val_step_perplexity

In [20]:
def get_preprocessed_dataset(tokenizer):
    # Load dataset from Hugging Face
    dataset = datasets.load_dataset("openai/gsm8k", 'main')
    dataset = dataset["train"]
    dataset = dataset.select(range(100))

    # Split the dataset into training and validation sets
    train_dataset, val_dataset = dataset.train_test_split(test_size=0.1, seed=42).values()  # Split 20% for validation

    def tokenize_add_label(sample):
        prompt = tokenizer.encode(tokenizer.bos_token + "###Input:\n" + sample["question"] + "\n", add_special_tokens=False)
        labels = tokenizer.encode("###Output:\n" + sample["answer"] + tokenizer.eos_token, add_special_tokens=False)
        sample = {
            "input_ids": prompt + labels,
            "attention_mask": [1] * (len(prompt) + len(labels)),
            "labels": prompt + labels
        }
        return sample

    # Apply the tokenize_add_label function to each dataset
    train_dataset = train_dataset.map(tokenize_add_label, remove_columns=list(train_dataset.features))
    val_dataset = val_dataset.map(tokenize_add_label, remove_columns=list(val_dataset.features))

    return train_dataset, val_dataset

In [21]:
@dataclass
class train_configy:
    model_name: str="openai-community/gpt2"
    tokenizer_name: str=None
    run_validation: bool=True
    batch_size_training: int=1
    batching_strategy: str="packing" #alternative: padding
    context_length: int=128
    gradient_accumulation_steps: int=1
    gradient_clipping: bool = True
    gradient_clipping_threshold: float = 1.0
    num_epochs: int=3
    max_train_step: int=0
    max_eval_step: int=0
    num_workers_dataloader: int=1
    lr: float=1e-4
    weight_decay: float=0.0
    gamma: float= 0.85
    seed: int=42
    mixed_precision: bool=True
    val_batch_size: int=1
    output_dir: str = "/content/test_model"
    save_model: bool = True
    save_metrics: bool = True # saves training metrics to a json file for later plotting
    flop_counter: bool = False # Enable flop counter to measure model throughput, can not be used with pytorch profiler at the same time.
    flop_counter_start: int = 3 # The step to start profiling, default is 3, which means after 3 steps of warmup stage, the profiler will start to count flops.
    use_profiler: bool = False # Enable pytorch profiler, can not be used with flop counter at the same time.
    profiler_dir: str = "/content/test_model" # will be used if using profiler

In [22]:
train_config = train_configy()

In [ ]:
dataset_train, dataset_val = get_preprocessed_dataset(
    tokenizer
)

In [24]:
print(f"--> Training Set Length = {len(dataset_train)}")
print(f"--> Validation Set Length = {len(dataset_val)}")

--> Training Set Length = 90
--> Validation Set Length = 10


In [25]:
if train_config.batching_strategy == "packing":
    dataset_train = ConcatDataset(dataset_train, chunk_size=train_config.context_length)

train_dl_kwargs = get_dataloader_kwargs(train_config, dataset_train, tokenizer, "train")

# Create DataLoaders for the training and validation dataset
train_dataloader = torch.utils.data.DataLoader(
    dataset_train,
    num_workers=train_config.num_workers_dataloader,
    pin_memory=True,
    **train_dl_kwargs,
)



Preprocessing dataset: 100%|██████████| 90/90 [00:00<00:00, 3587.06it/s]


In [26]:
print(type(train_dataloader))
print(train_dataloader)

<class 'torch.utils.data.dataloader.DataLoader'>


In [27]:
eval_dataloader = None
if train_config.run_validation:
    if train_config.batching_strategy == "packing":
        dataset_val = ConcatDataset(dataset_val, chunk_size=train_config.context_length)

    val_dl_kwargs = get_dataloader_kwargs(train_config, dataset_val, tokenizer, "val")

    eval_dataloader = torch.utils.data.DataLoader(
        dataset_val,
        num_workers=train_config.num_workers_dataloader,
        pin_memory=True,
        **val_dl_kwargs,
    )

Preprocessing dataset: 100%|██████████| 10/10 [00:00<00:00, 1437.24it/s]


In [28]:
optimizer = optim.AdamW(
        model.parameters(),
        lr=train_config.lr,
        weight_decay=train_config.weight_decay,
    )
scheduler = StepLR(optimizer, step_size=1, gamma=train_config.gamma)

In [29]:
# Start the training process
results = train(
    model,
    train_dataloader,
    eval_dataloader,
    tokenizer,
    optimizer,
    scheduler,
    train_config.gradient_accumulation_steps,
    train_config,
    None,
)
[print(f'Key: {k}, Value: {v}') for k, v in results.items()]

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:343: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
Training Epoch: 1:   0%|          | 0/115 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Training Epoch: 1/3, step 0/115 completed (loss: 3.5776889324188232):   1%|          | 1/115 [00:01<01:57,  1.03s/it]
evaluating Epoch: 100%|██████████| 14/14 [00:00<00:00, 16.30it/s]
Training Epoch: 1/3, step 50/115 completed (loss: 2.4937636852264404):  44%|████▍     | 51/115 [00:08<00:07,  8.63it/s]
evaluating Epoch: 100%|██████████| 14/14 [00:01<00:00, 12.33it/s]
Training Epoch: 1/3, step 100/115 completed (loss: 2.2221696376800537):  88%|████████▊ | 101/115 [00:17<00:01,  7.06it/s

Max CUDA memory allocated was 2 GB
Max CUDA memory reserved was 2 GB
Peak active CUDA memory was 2 GB
CUDA Malloc retries : 0
CPU Total Peak Memory consumed during the train (max): 2 GB


evaluating Epoch: 100%|██████████| 14/14 [00:00<00:00, 24.51it/s]


Model is saved in /content/test_model/epoch0 directory
best eval loss on epoch 1 is 2.23980450630188
Epoch 1: train_perplexity=13.6916, train_epoch_loss=2.6168, epoch time 21.769388803000027s


/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:343: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
Training Epoch: 2:   0%|          | 0/115 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Training Epoch: 2/3, step 0/115 completed (loss: 1.7043362855911255):   1%|          | 1/115 [00:00<00:19,  5.79it/s]
evaluating Epoch: 100%|██████████| 14/14 [00:00<00:00, 15.48it/s]
Training Epoch: 2/3, step 50/115 completed (loss: 1.7790257930755615):  44%|████▍     | 51/115 [00:08<00:07,  8.20it/s]
evaluating Epoch: 100%|██████████| 14/14 [00:00<00:00, 16.91it/s]
Training Epoch: 2/3, step 100/115 completed (loss: 1.5589760541915894):  88%|████████▊ | 101/115 [00:15<00:01,  8.43it/s

...

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:343: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
Training Epoch: 3:   0%|          | 0/115 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Training Epoch: 3/3, step 0/115 completed (loss: 1.2424379587173462):   1%|          | 1/115 [00:00<00:20,  5.60it/s]
evaluating Epoch: 100%|██████████| 14/14 [00:00<00:00, 16.11it/s]
Training Epoch: 3/3, step 50/115 completed (loss: 1.3549705743789673):  44%|████▍     | 51/115 [00:07<00:08,  7.91it/s]
evaluating Epoch: 100%|██████████| 14/14 [00:01<00:00, 13.59it/s]
Training Epoch: 3/3, step 100/115 completed (loss: 1.086852788925171):  88%|████████▊ | 101/115 [00:15<00:01,  8.72it/s]

Max CUDA memory allocated was 2 GB
Max CUDA memory reserved was 2 GB
Peak active CUDA memory was 2 GB
CUDA Malloc retries : 0
CPU Total Peak Memory consumed during the train (max): 2 GB


evaluating Epoch: 100%|██████████| 14/14 [00:00<00:00, 24.15it/s]


Model is saved in /content/test_model/epoch2 directory
Epoch 3: train_perplexity=3.8314, train_epoch_loss=1.3432, epoch time 19.770632068999987s
Key: avg_train_prep, Value: 7.767048199971517
Key: avg_train_loss, Value: 1.9046973784764607
Key: avg_eval_prep, Value: 10.426742553710938
Key: avg_eval_loss, Value: 2.23980450630188
Key: avg_epoch_time, Value: 20.587568546666677
Key: avg_checkpoint_time, Value: 2.593422291333335
Key: metrics_filename, Value: /content/test_model/metrics_data_None-2024-09-19_23-06-35.json


[None, None, None, None, None, None, None]

###Inference from Trained Model

In [30]:
del model

In [31]:
model = AutoModelForCausalLM.from_pretrained(
    "/content/test_model/epoch2",
    device_map="cuda",
    use_cache=None,
    attn_implementation=None,
)

In [32]:
user_prompt = '''What is the sum of 20 + 30?'''
batch = tokenizer(user_prompt, return_tensors="pt")

In [33]:
batch = {k: v.to("cuda") for k, v in batch.items()}
with torch.no_grad():
    outputs = model.generate(
        **batch,
        max_new_tokens=50, #The maximum numbers of tokens to generate
        do_sample=False, #Whether or not to use sampling ; use greedy decoding otherwise.
        top_p=1.0, # [optional] If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
        temperature=0, # [optional] The value used to modulate the next token probabilities.
        min_length=None, #The minimum length of the sequence to be generated, input prompt + min_new_tokens
        use_cache=True, #[optional] Whether or not the model should use the past last key/values attentions Whether or not the model should use the past last key/values attentions (if applicable to the model) to speed up decoding.
        top_k=50, # [optional] The number of highest probability vocabulary tokens to keep for top-k-filtering.
        repetition_penalty=1.0, #The parameter for repetition penalty. 1.0 means no penalty.
        length_penalty=1, #[optional] Exponential penalty to the length that is used with beam-based generation.
        output_hidden_states= True, return_dict_in_generate=True,
    )
batch = {k: v.to("cpu") for k, v in batch.items()}

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [34]:
output_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print("The complete decode:",output_text)
print("--------------------------------")
print("Just the model's response",output_text[len(user_prompt):])

The complete decode: What is the sum of 20 + 30?
###Output:
James earns $20 an hour while working his shift.  He earns $15 an hour while working his shift.  How much does he earn per week?
###Output:
James earns $20 an hour while working
--------------------------------
Just the model's response 
###Output:
James earns $20 an hour while working his shift.  He earns $15 an hour while working his shift.  How much does he earn per week?
###Output:
James earns $20 an hour while working
